# Module import

In [27]:
import pandas as pd
import numpy as np

# Read the data

For now, the data are read directly in the directory where a copy of the spreadsheet has been put (in github directory). In the future, this will need to change

In [28]:
df = pd.read_excel('BDD consolidée - toutes les tables_v03.xlsx')#, dtype={'Groupe': str})

xls = pd.ExcelFile('BDD consolidée - toutes les tables_v03.xlsx')
df_companies_data = pd.read_excel(xls, 'Companies data')
df_companies_brands = pd.read_excel(xls, 'Companies brands')
df_companies_competitors = pd.read_excel(xls, 'Companies competitors')
df_global_score = pd.read_excel(xls, 'Global score display')
df_emissions_scope = pd.read_excel(xls, 'Emissions scopes description')
df_direct_complete_score = pd.read_excel(xls, 'Direct and complete score displ')
df_direct_commitment = pd.read_excel(xls, 'Direct and complete commitment ')
df_coeff_director = pd.read_excel(xls, 'Coeff directeur pour graph')
df_pos_cursors = pd.read_excel(xls, 'Positionnement des curseurs')
df_marques_by_cat = pd.read_excel(xls, 'Table Marques par catégorie')


## Brands: names, id and logo

In [29]:
df_final = df_companies_data.copy(deep=True)
list_groups = df["Group"]
df_final.columns

Index(['Group', 'Sector', 'Logo', 'Country code', 'Turnover',
       'Climate website', 'Comment', 'Global score', 'C1 direct score',
       'C1 reduction', 'C1 unit', 'C1 initial date', 'C1 final date',
       'C2 complete score', 'C2 reduction', 'C2 unit', 'C2 initial date',
       'C2 final date', 'Footnote', 'E1 score commitment direct', 'E1 phrase',
       'E1 perimeter', 'E1 reduction', 'E1 unit', 'E1 final date',
       'E1 initial date', 'E2 score commitment', 'E2 phrase', 'E2 perimeter',
       'E2 reduction', 'E2 unit', 'E2 final date', 'E2 initial date',
       'Share direct vs complet', 'Total emissions', 'Cat 1 amount',
       'Cat 1 name', 'Cat 2 amount', 'Cat 2 name', 'Cat 3 amount',
       'Cat 3 name', 'Cat 4 amount', 'Cat 4 name', 'Cat 5 amount',
       'Cat 5 name', 'Cat 6 amount', 'Cat 6 name'],
      dtype='object')

In [30]:
import base64

def encode(text):
  btext = text.encode('utf-8')[:6]
  return base64.b64encode(btext).decode("utf-8") 


df_final['company_id'] = [encode(x) for x in df_final['Group']]

In [31]:
top_5_brands_names=[]
top_5_brands_logos=[]

# Do a loop over the list of groups and filter the table df_top_brands on the group name
# Keep only the first 5 groups
# If less than 5 groups, print 'NaN' instead

for (k,group) in enumerate(list_groups):
    df_top_brands = df_companies_brands[df_companies_brands["Groupe"]==group].sort_values("Ordre")[:5]

    group_5_brand_name=[]
    group_5_brand_logo=[]
    
    for i in range(5):
        if i<len(df_top_brands):
            group_5_brand_name.append(df_top_brands["Marques"].iloc[i])
            group_5_brand_logo.append(df_top_brands["Logo"].iloc[i])
           
        else:
            group_5_brand_name.append('NaN')
            group_5_brand_logo.append('NaN')

    top_5_brands_names.append(group_5_brand_name)    
    top_5_brands_logos.append(group_5_brand_logo)    


df_final["top_brands"] = top_5_brands_names
df_final[["Logo brand 1","Logo brand 2","Logo brand 3","Logo brand 4","Logo brand 5"]] = top_5_brands_logos

In [32]:
top_5_brands_names

[['Andros', 'Bonne Maman', 'Le berger des fruits', 'Nova', 'Pierrot Gourmand'],
 ['Allinson', 'Ryvita', 'Argo', 'Silver Spoon', 'Askeys'],
 ['Babybel', 'Boursin', 'Kiri', 'Leerdammer', 'La vache qui rit'],
 ['Actimel', 'Activia', 'Alpro', 'Aqua', 'Badoit'],
 ["Abbot kinney's", 'Allos', 'Alter eco', 'Bjorg', 'Bonneterre'],
 ['Delacre', 'Duplo', 'Ferrero Rocher', 'Kinder', 'Mon Chéri'],
 ['Haagen-Dazs', 'Old El Paso', 'Yoplait', 'NaN', 'NaN'],
 ['Charles & Alice', 'NaN', 'NaN', 'NaN', 'NaN'],
 ['Coco pops', 'Country store', 'Froot Loops', 'Frosties', "Kellogg's"],
 [5.0, '3 musketeers', 'Altoids', 'American heritage', 'Amicelli'],
 ['Belvita', 'Cadbury', "Côté d'or", 'Daim', 'Hollywood'],
 ['After eight', 'BabyNes', 'Buitoni', 'Caro', 'Cat Chow'],
 ['7up', 'Aquafina', 'Cheetps', 'Doritos', 'Frappuccino Strabuck Coffee'],
 ['Bref', 'Catch', 'Ceresit', "Croc'odor", "Décap'four"],
 ['always', 'Ambi Pur', 'Antikal', 'Apaisyl', 'Ariel'],
 ['Axe', "Ben & Jerry's", 'Brut', 'Cajoline', "Carte d'

## Competitors : names

In [33]:
# Just need to rename the column to match the column of final dataframe and do a merge
df_companies_competitors.rename(columns={"Groupe":"Group"}, inplace=True)
df_final = pd.merge(df_final, df_companies_competitors, how="left", on="Group")

## Comments

In [34]:
# TODO, this should come from NLP analysis

## Actual Scores

### Pre-formatting 

#### Coeff_director

In [35]:
# format the table so that a multi-index gets back the coefficient directly
df_coeff_director = df_coeff_director.drop(columns=["degre final"])
df_coeff_director = df_coeff_director.set_index(["Value type","Courbes"])
df_coeff_director

Coeff directeur
Value type          Courbes                 
en valeur absolue   2°C              -0.0125
                    1,8°C            -0.0250
                    1,5°C            -0.0420
par unité de volume 2°C              -0.0200
                    1,8°C            -0.0400
                    1,5°C            -0.0672
par unité de valeur 2°C              -0.0700
                    1,8°C            -0.1000
                    1,5°C            -0.1500

#### Cursor position

In [36]:
df_pos_cursors = df_pos_cursors.drop(columns=["Catégorie du secteur","Intervalle","Commentaire","level"]).rename(columns={"Value type - Grandeur utilisée (évolution annuelle CO2)":"unit",
                "Borne max (évolution annuelle CO2)":"max",
                "Borne min (évolution annuelle CO2)":"min"})

df_pos_cursors = df_pos_cursors.set_index(["unit","Score"])

df_pos_cursors

max     min
unit                Score                
en valeur absolue   6.0   -0.0420 -0.0840
                    5.0   -0.0250 -0.0420
                    4.0   -0.0125 -0.0250
                    3.0    0.0000 -0.0125
                    2.0    0.0420  0.0000
                    99.0      NaN     NaN
                    1.0       NaN     NaN
par unité de volume 6.0   -0.0672 -0.1344
                    5.0   -0.0400 -0.0672
                    4.0   -0.0200 -0.0400
                    3.0    0.0000 -0.0200
                    2.0    0.0672  0.0000
                    99.0      NaN     NaN
                    1.0       NaN     NaN
par unité de valeur 6.0   -0.1500 -0.3000
                    5.0   -0.1000 -0.1500
                    4.0   -0.0700 -0.1000
                    3.0    0.0000 -0.0700
                    2.0    0.1500  0.0000
                    99.0      NaN     NaN
                    1.0       NaN     NaN

#### Display of scores

In [37]:
df_global_score = df_global_score.set_index("Global score")
df_global_score

,Color Hex,Short label,Long label,Badge
Global score,,,,
1.0,820000.0,Undisclosed,Carbon footprint not disclosed,NaN
2.0,C00000,Highly insufficient,Increasing its carbon footprint,NaN
3.0,FF8939,Insufficient,Stable ou slowly decreasing carbon footprint,NaN
4.0,FEC800,Partial,Carbon footprint partially decreasing,NaN
5.0,8CDF41,Strong,Carbon footprint strongly decreasing,NaN
6.0,0DB800,Very strong,Carbon footprint very strongly decreasing,NaN


In [38]:
df_direct_complete_score = df_direct_complete_score.set_index("Direct or complete score")
df_direct_complete_score

,Color Hex,Short label,Long label
Direct or complete score,,,
1.0,820000.0,Not reported,Not disclosed
2.0,C00000,Towards +4°C,Increasing emissions. Heading towards +4°C.
3.0,FF8939,Between +2°C and +3°C,Stable ou decreasing too slowly. Not compatibl...
4.0,FEC800,2°C,Slightly decreasing. Compatible 2°C.
5.0,8CDF41,Well-below 2°C,Strongly decreasing. Compatible well-below 2°C
6.0,0DB800,1.5°C,Very strongly decreasing. Compatible 1.5°C.
99.0,C00000,n.a. (too recent),Measure only recent
n.a.,n.a.,n.a.,n.a.


In [39]:
df_direct_commitment = df_direct_commitment.set_index("Direct or complete score")
df_direct_commitment

,Color Hex,Short label,Long label
Direct or complete score,,,
1.0,820000.0,Not reported,Not reported
2.0,C00000,Towards +4°C,Towards +4°C
3.0,FF8939,Between +2°C and +3°C,Ambition not compatible 2°C
4.0,FEC800,2°C,Ambition compatible 2°C
5.0,8CDF41,Well-below 2°C,Ambition compatible Well-below 2°C
6.0,0DB800,1.5°C,"Ambition compatible 1,5°C"
99.0,BFBFBF,n.a. (too recent),No commitment disclosed
n.a.,n.a.,n.a.,na.


### Global score

In [40]:
# This assumes that there is always a value for global score
df_final[["global_score_hexa_color_code","global_score_short_label"]] = df_final["Global score"].apply(lambda x: df_global_score.loc[x])[["Color Hex", "Short label"]]

# TODO : logo

### Direct score

In [41]:
# This computes the various percentages depending on the scenarios along with the cursor level
C1_perc_scenarios = []
C1_direct_level = []
C1_reduc_per_year=[]
C1_final_value = []
for (k,group) in enumerate(list_groups):
    ini_date = df_final.loc[k,"C1 initial date"]
    fin_date = df_final.loc[k,"C1 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        
        if (df_final.loc[k,"C1 reduction"] != "n.a."):
            C1_final_value.append(100.0 + 100.0*df_final.loc[k,"C1 reduction"])
        
        
        # percentage per scenarios
        dim_perc_2 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '2°C'])
        dim_perc_18 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '1,8°C'])
        dim_perc_15 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '1,5°C'])
        C1_perc_scenarios.append([100.0+dim_perc_2,100.0+dim_perc_18,100.0+dim_perc_15])
        
        # compute direct level of the cursor
        reduc_per_year = df_final.loc[k,"C1 reduction"] / diff_year
        min_inter = float(df_pos_cursors.loc[df_final.loc[k,"C1 unit"], df_final.loc[k,"C1 direct score"]]["min"])
        max_inter = float(df_pos_cursors.loc[df_final.loc[k,"C1 unit"], df_final.loc[k,"C1 direct score"]]["max"])
        C1_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; C1 score in DBB : ",df_final.loc[k,"C1 direct score"],
                 "but the annual reduction is: ", 100.0*reduc_per_year)
            
        
        if df_final.loc[k,"C1 direct score"] == 6.0 and reduc_per_year<min_inter:
            C1_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"C1 direct score"] == 2.0 and reduc_per_year>max_inter:
            C1_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            C1_direct_level.append(df_final.loc[k,"C1 direct score"] + (reduc_per_year-max_inter)/(min_inter - max_inter))
    else:
        C1_perc_scenarios.append(["n.a.","n.a.","n.a."])
        C1_direct_level.append("n.a.")
        C1_reduc_per_year.append("n.a.")
        C1_final_value.append("n.a.")


df_final[["C1 2deg final","C1 1,8deg final","C1 1,5deg final"]] = C1_perc_scenarios
df_final["C1 reduc per year"] = C1_reduc_per_year
df_final["C1 direct level"] = C1_direct_level
df_final["C1 final value"] = C1_final_value


# This assumes that there is always a value for direct score and computes the display related variables
df_final[["direct_score_hexa_color_code","direct_score_short_label"]] = df_final["C1 direct score"].apply(lambda x: df_direct_complete_score.loc[x])[["Color Hex", "Short label"]]


problem with  Ecotone ; C1 score in DBB :  6.0 but the annual reduction is:  -4.066543676425535
problem with  Ferrero ; C1 score in DBB :  2.0 but the annual reduction is:  6.920748524024988
problem with  General Mills ; C1 score in DBB :  6.0 but the annual reduction is:  -4.195823573659525
problem with  Unilever ; C1 score in DBB :  6.0 but the annual reduction is:  -10.743296601975
problem with  Coty ; C1 score in DBB :  6.0 but the annual reduction is:  -11.367949999999999
problem with  L'Oréal ; C1 score in DBB :  6.0 but the annual reduction is:  -9.913487699999997
problem with  Beiersdorf ; C1 score in DBB :  6.0 but the annual reduction is:  -14.548000799999993


### C2 columns

In [42]:
# This computes the various percentages depending on the scenarios along with the cursor level
C2_perc_scenarios = []
C2_direct_level = []
C2_reduc_per_year=[]
C2_final_value = []
for (k,group) in enumerate(list_groups):
    ini_date = df_final.loc[k,"C2 initial date"]
    fin_date = df_final.loc[k,"C2 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        
        if (df_final.loc[k,"C2 reduction"] != "n.a."):
            C2_final_value.append(100.0 + 100.0*df_final.loc[k,"C2 reduction"])
        
        
        # percentage per scenarios
        dim_perc_2 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '2°C'])
        dim_perc_18 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '1,8°C'])
        dim_perc_15 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '1,5°C'])
        C2_perc_scenarios.append([100.0+dim_perc_2,100.0+dim_perc_18,100.0+dim_perc_15])
        
        # compute direct level of the cursor
        reduc_per_year = df_final.loc[k,"C2 reduction"] / diff_year
        min_inter = float(df_pos_cursors.loc[df_final.loc[k,"C2 unit"], df_final.loc[k,"C2 complete score"]]["min"])
        max_inter = float(df_pos_cursors.loc[df_final.loc[k,"C2 unit"], df_final.loc[k,"C2 complete score"]]["max"])
        C2_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; C2 score in DBB : ",df_final.loc[k,"C2 complete score"],
                 "but the annual reduction is: ", 100.0*reduc_per_year)
            
        
        if df_final.loc[k,"C2 complete score"] == 6.0 and reduc_per_year<min_inter:
            C2_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"C2 complete score"] == 2.0 and reduc_per_year>max_inter:
            C2_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            C2_direct_level.append(df_final.loc[k,"C2 complete score"] + (reduc_per_year-max_inter)/(min_inter - max_inter))
    else:
        C2_perc_scenarios.append(["n.a.","n.a.","n.a."])
        C2_direct_level.append("n.a.")
        C2_reduc_per_year.append("n.a.")
        C2_final_value.append("n.a.")


df_final[["C2 2deg final","C2 1,8deg final","C2 1,5deg final"]] = C2_perc_scenarios
df_final["C2 reduc per year"] = C2_reduc_per_year
df_final["C2 complete level"] = C2_direct_level
df_final["C2 final value"] = C2_final_value

# This assumes that there is always a value for complete score and computes the display related variables
df_final[["complete_score_hexa_color_code","complete_score_short_label"]] = df_final["C2 complete score"].apply(lambda x: df_direct_complete_score.loc[x])[["Color Hex", "Short label"]]


problem with  Bel ; C2 score in DBB :  5.0 but the annual reduction is:  -7.300000000000001
problem with  Coty ; C2 score in DBB :  3.0 but the annual reduction is:  0.10000000000000009


In [43]:
df_final.columns

Index(['Group', 'Sector', 'Logo', 'Country code', 'Turnover',
       'Climate website', 'Comment', 'Global score', 'C1 direct score',
       'C1 reduction', 'C1 unit', 'C1 initial date', 'C1 final date',
       'C2 complete score', 'C2 reduction', 'C2 unit', 'C2 initial date',
       'C2 final date', 'Footnote', 'E1 score commitment direct', 'E1 phrase',
       'E1 perimeter', 'E1 reduction', 'E1 unit', 'E1 final date',
       'E1 initial date', 'E2 score commitment', 'E2 phrase', 'E2 perimeter',
       'E2 reduction', 'E2 unit', 'E2 final date', 'E2 initial date',
       'Share direct vs complet', 'Total emissions', 'Cat 1 amount',
       'Cat 1 name', 'Cat 2 amount', 'Cat 2 name', 'Cat 3 amount',
       'Cat 3 name', 'Cat 4 amount', 'Cat 4 name', 'Cat 5 amount',
       'Cat 5 name', 'Cat 6 amount', 'Cat 6 name', 'company_id', 'top_brands',
       'Logo brand 1', 'Logo brand 2', 'Logo brand 3', 'Logo brand 4',
       'Logo brand 5', 'Competitor 1', 'Competitor 2', 'Competitor 3',
    

In [44]:
df_final[['C2 reduction', 'C1 direct level']]

,C2 reduction,C1 direct level
0,n.a.,n.a.
1,n.a.,5.376597
2,-0.146,3.78808
3,-0.275,6.806615
4,n.a.,5.968225
5,0.075,2.0
6,-0.17,5.999006
7,n.a.,4.779685
8,-0.048,6.706647
9,-0.02,6.305066


In [45]:
df_final.loc[2,"C2 reduction"]

-0.14600000000000002

# Commitments scores

## E1 columns

In [46]:
# This computes the various percentages depending on the scenarios along with the cursor level
E1_direct_level = []
E1_reduc_per_year=[]
E1_final_value = []

for (k,group) in enumerate(list_groups):

    ini_date = df_final.loc[k,"E1 initial date"]
    fin_date = df_final.loc[k,"E1 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        if (df_final.loc[k,"E1 reduction"] != "n.a."):
            E1_final_value.append(100.0 - float(df_final.loc[k,"E1 reduction"].split('%')[0]))
        
        
        # compute direct level of the cursor
        reduc_per_year = -float(df_final.loc[k,"E1 reduction"].split('%')[0]) / diff_year
        min_inter = 100.0*float(df_pos_cursors.loc[df_final.loc[k,"E1 unit"], df_final.loc[k,"E1 score commitment direct"]]["min"])
        max_inter = 100.0*float(df_pos_cursors.loc[df_final.loc[k,"E1 unit"], df_final.loc[k,"E1 score commitment direct"]]["max"])
        E1_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; E1 score in DBB : ",df_final.loc[k,"E1 score commitment direct"],
                 "but the annual reduction is: ", reduc_per_year,"; and interval is: ",min_inter,max_inter)
            
        
        if df_final.loc[k,"E1 score commitment direct"] == 6.0 and reduc_per_year<min_inter:
            E1_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"E1 score commitment direct"] == 2.0 and reduc_per_year>max_inter:
            E1_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            E1_direct_level.append(df_final.loc[k,"E1 score commitment direct"] + 
                                   (reduc_per_year-max_inter)/(min_inter - max_inter))
         
    else:
        E1_direct_level.append("n.a.")
        E1_reduc_per_year.append("n.a.")
        E1_final_value.append("n.a.")


df_final["E1 reduc per year"] = E1_reduc_per_year
df_final["E1 direct level"] = E1_direct_level
df_final["E1 final value"] = E1_final_value


# This assumes that there is always a value for direct score and computes the display related variables
df_final[["direct_ambition_hexa_color_code","direct_ambition_long_label"]] = df_final["E1 score commitment direct"].apply(lambda x: df_direct_commitment.loc[x])[["Color Hex", "Short label"]]


problem with  Bel ; E1 score in DBB :  5.0 but the annual reduction is:  -1.6923076923076923 ; and interval is:  -4.2 -2.5
problem with  Henkel ; E1 score in DBB :  6.0 but the annual reduction is:  -5.153846153846154 ; and interval is:  -13.44 -6.72
problem with  Unilever ; E1 score in DBB :  4.0 but the annual reduction is:  -6.666666666666667 ; and interval is:  -2.5 -1.25
problem with  Coty ; E1 score in DBB :  5.0 but the annual reduction is:  -2.3076923076923075 ; and interval is:  -4.2 -2.5
problem with  L'Oréal ; E1 score in DBB :  6.0 but the annual reduction is:  -11.11111111111111 ; and interval is:  -8.4 -4.2
problem with  Beiersdorf ; E1 score in DBB :  6.0 but the annual reduction is:  -2.5 ; and interval is:  -8.4 -4.2


### E2 columns

In [47]:
# This computes the various percentages depending on the scenarios along with the cursor level
E2_direct_level = []
E2_reduc_per_year=[]
E2_final_value = []

for (k,group) in enumerate(list_groups):

    ini_date = df_final.loc[k,"E2 initial date"]
    fin_date = df_final.loc[k,"E2 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        if (df_final.loc[k,"E2 reduction"] != "n.a."):
            E2_final_value.append(100.0 - float(df_final.loc[k,"E2 reduction"].split('%')[0]))
        
        
        # compute direct level of the cursor
        reduc_per_year = -float(df_final.loc[k,"E2 reduction"].split('%')[0]) / diff_year
        min_inter = 100.0*float(df_pos_cursors.loc[df_final.loc[k,"E2 unit"], df_final.loc[k,"E2 score commitment"]]["min"])
        max_inter = 100.0*float(df_pos_cursors.loc[df_final.loc[k,"E2 unit"], df_final.loc[k,"E2 score commitment"]]["max"])
        E2_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; E2 score in DBB : ",df_final.loc[k,"E2 score commitment"],
                 "but the annual reduction is: ", reduc_per_year,"; and interval is: ",min_inter,max_inter)
            
        
        if df_final.loc[k,"E2 score commitment"] == 6.0 and reduc_per_year<min_inter:
            E2_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"E2 score commitment"] == 2.0 and reduc_per_year>max_inter:
            E2_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            E2_direct_level.append(df_final.loc[k,"E2 score commitment"] + 
                                   (reduc_per_year-max_inter)/(min_inter - max_inter))
         
    else:
        E2_direct_level.append("n.a.")
        E2_reduc_per_year.append("n.a.")
        E2_final_value.append("n.a.")


df_final["E2 reduc per year"] = E2_reduc_per_year
df_final["E2 complete level"] = E2_direct_level
df_final["E2 final value"] = E2_final_value


# This assumes that there is always a value for direct score and computes the display related variables
df_final[["complete_ambition_hexa_color_code","complete_ambition_long_label"]] = df_final["E2 score commitment"].apply(lambda x: df_direct_commitment.loc[x])[["Color Hex", "Short label"]]


problem with  Danone ; E2 score in DBB :  5.0 but the annual reduction is:  -3.3333333333333335 ; and interval is:  -6.72 -4.0
problem with  Ferrero ; E2 score in DBB :  5.0 but the annual reduction is:  -3.5833333333333335 ; and interval is:  -6.72 -4.0
problem with  Colgate-Palmolive ; E2 score in DBB :  5.0 but the annual reduction is:  -4.285714285714286 ; and interval is:  -4.2 -2.5
problem with  Coty ; E2 score in DBB :  5.0 but the annual reduction is:  -2.3076923076923075 ; and interval is:  -4.2 -2.5
problem with  L'Oréal ; E2 score in DBB :  5.0 but the annual reduction is:  -3.5714285714285716 ; and interval is:  -6.72 -4.0
problem with  Beiersdorf ; E2 score in DBB :  4.0 but the annual reduction is:  -0.8333333333333334 ; and interval is:  -2.5 -1.25
problem with  GSK ; E2 score in DBB :  4.0 but the annual reduction is:  -1.2307692307692308 ; and interval is:  -2.5 -1.25


## Category

In [52]:
df_marques_by_cat

,Marque,Categorie,Logo marque
0,Bjorg,Biscuits et gâteaux,https://firebasestorage.googleapis.com/v0/b/st...
1,Bonne Maman,Biscuits et gâteaux,https://firebasestorage.googleapis.com/v0/b/st...
2,Cadbury,Biscuits et gâteaux,https://firebasestorage.googleapis.com/v0/b/st...
3,Delacre,Biscuits et gâteaux,https://firebasestorage.googleapis.com/v0/b/st...
4,Kinder,Biscuits et gâteaux,https://firebasestorage.googleapis.com/v0/b/st...
...,...,...,...
155,Fairy,Nettoyants vaisselle,https://firebasestorage.googleapis.com/v0/b/st...
156,Finish,Nettoyants vaisselle,https://firebasestorage.googleapis.com/v0/b/st...
157,Mir,Nettoyants vaisselle,https://firebasestorage.googleapis.com/v0/b/st...
158,Paic,Nettoyants vaisselle,https://firebasestorage.googleapis.com/v0/b/st...


# Renaming and final modifications

In [48]:
df_data_viz = df_final.copy(deep=True)
df_data_viz.columns

Index(['Group', 'Sector', 'Logo', 'Country code', 'Turnover',
       'Climate website', 'Comment', 'Global score', 'C1 direct score',
       'C1 reduction', 'C1 unit', 'C1 initial date', 'C1 final date',
       'C2 complete score', 'C2 reduction', 'C2 unit', 'C2 initial date',
       'C2 final date', 'Footnote', 'E1 score commitment direct', 'E1 phrase',
       'E1 perimeter', 'E1 reduction', 'E1 unit', 'E1 final date',
       'E1 initial date', 'E2 score commitment', 'E2 phrase', 'E2 perimeter',
       'E2 reduction', 'E2 unit', 'E2 final date', 'E2 initial date',
       'Share direct vs complet', 'Total emissions', 'Cat 1 amount',
       'Cat 1 name', 'Cat 2 amount', 'Cat 2 name', 'Cat 3 amount',
       'Cat 3 name', 'Cat 4 amount', 'Cat 4 name', 'Cat 5 amount',
       'Cat 5 name', 'Cat 6 amount', 'Cat 6 name', 'company_id', 'top_brands',
       'Logo brand 1', 'Logo brand 2', 'Logo brand 3', 'Logo brand 4',
       'Logo brand 5', 'Competitor 1', 'Competitor 2', 'Competitor 3',
    

In [49]:
df_data_viz = df_data_viz.rename(columns={
    "Group": "company_name",
    "Logo": "",
    "Sector": "sector",
    "C1 final value": "c1_final_value",
    "C1 2deg final": "c1_2deg_final",
    "C1 1,8deg final": "c1_1_8deg_final",
    "C1 1,5deg final": "c1_1_5_deg_final",
    "C1 initial date": "C1_initial_date",
    "C1 final date": "C1_final_date",
    "C2 final value": "c2_final_value",
    "C2 2deg final": "c2_2deg_final",
    "C2 1,8deg final": "c2_1_8deg_final",
    "C2 1,5deg final": "c2_1_5deg_final",
    "C2 initial date": "C2_initial_date",
    "C2 final date": "C2_final_date",
    "Comment": "comment",
    "Global score": "global_score",
    "C1 direct score": "direct_score",
    "C2 complete score": "complete_score",
    "E1 score commitment direct": "direct_rounding_score_commitments",
    "E1 direct level": "direct_score_commitments",
    "E1 phrase": "direct_commitments_sentence",
    "E2 score commitment": "complete_rounding_score_commitments",
    "E2 complete level": "complete_score_commitments",

    "E2 phrase": "complete_commitments_sentence",
    "Total emissions": "total_emissions",
    "Country code": "country_flag"
})

In [50]:
list_names_var_dataviz = ["company_name","company_id","c1_final_value","c1_2deg_final","c1_1_8deg_final",
    "c1_1_5_deg_final","c2_final_value","c2_2deg_final","c2_1_8deg_final","c2_1_5deg_final","C1_initial_date",
    "C1_final_date","C2_initial_date","C2_final_date","global_score_hexa_color_code","global_score_short_label","comment",
    "global_score","global_score_logo_path","direct_score","direct_score_hexa_color_code","direct_score_short_label",
    "complete_score","complete_score_hexa_color_code","complete_score_short_label","sector",
    "direct_commitments_sentence","complete_commitments_sentence","direct_ambition_long_label","complete_ambition_long_label",
    "direct_rounding_score_commitments","complete_rounding_score_commitments","direct_score_commitments",
    "complete_score_commitments","direct_ambition_hexa_color_code","complete_ambition_hexa_color_code",
    "total_emissions","total_emissions_year","emissions_category_amount","emissions_category_name",
    "emissions_category_hover","brand_logo","country_flag","top_brands","revenue","revenue_year"
]

In [51]:
df_data_viz[list_names_var_dataviz]

KeyError: "['global_score_logo_path', 'total_emissions_year', 'emissions_category_amount', 'emissions_category_name', 'emissions_category_hover', 'brand_logo', 'country_flag', 'revenue', 'revenue_year'] not in index"

In [78]:
df_final

,Group,Sector,Logo,Country code,Turnover,Climate website,Comment,Global score,C1 direct score,C1 reduction,...,"C2 1,8deg final","C2 1,5deg final",C2 reduc per year,C2 complete level,C2 final value,complete_score_hexa_color_code,complete_score_short_label,E1 reduc per year,E1 direct level,E1 final value
0,Andros,Food,NaN,FR,"2,5 Mrd € (est. 2020)",https://www.andros.fr/nos-engagements/,Un culte du secret incompatible avec les enjeu...,1.0,1.0,n.a.,...,n.a.,n.a.,n.a.,n.a.,n.a.,820000.0,Not reported,n.a.,n.a.,n.a.
1,Associated British Foods (alim.),Food,NaN,UK,4 Mrd € (2020) (groupe complet 16 Mrd €),https://www.abf.co.uk/responsibility,Seuls ~10% de son empreinte carbone complète s...,3.0,5.0,-0.125609,...,n.a.,n.a.,n.a.,n.a.,n.a.,820000.0,Not reported,n.a.,n.a.,n.a.
2,Bel,Food,NaN,FR,"3,5 Mrd € (2020)",https://www.groupe-bel.com/fr/nos-engagements/...,"Réduction encore partielle, malgré une forte r...",4.0,3.0,-0.039404,...,92.0,86.56,-7.3,6.213235,85.4,8CDF41,Well-below 2°C,-169.230769,4.524887,78.0
3,Danone,Food,NaN,FR,25 Mrd € (2019),https://www.danone.com/fr/impact/planet/toward...,Très fortes réductions de toutes ses émissions...,6.0,6.0,-0.379389,...,80.0,66.4,-5.5,5.551471,72.5,8CDF41,Well-below 2°C,-200.0,4.6,70.0
4,Ecotone,Food,NaN,NL,650 Mio € (2019),https://www.ecotone.bio/fr/autres-engagements/,Très forte réduction de ses émissions propres....,5.0,6.0,-0.325323,...,n.a.,n.a.,n.a.,n.a.,n.a.,820000.0,Not reported,-423.076923,6.007326,45.0
5,Ferrero,Food,NaN,IT,11 Mrd € (2019),https://www.ferrerosustainability.com/int/en/o...,"Des émissions toujours en hausse. Depuis 2010,...",2.0,2.0,0.27683,...,92.5,87.4,2.5,2.404762,107.5,C00000,Towards +4°C,-416.666667,5.980392,50.0
6,General Mills,Food,NaN,US,15 Mrd € (2020),https://www.generalmills.com/en/Responsibility...,Haut niveau d'action initié. A même renforcé s...,5.0,6.0,-0.209791,...,75.0,58.0,-1.7,4.36,83.0,FEC800,2°C,-420.0,6.0,58.0
7,Hero,Food,NaN,CH,1 Mrd € (2020),https://hero-group.ch/sustainability,Réduction significative de ses émissions propr...,4.0,4.0,-0.320343,...,n.a.,n.a.,n.a.,n.a.,n.a.,820000.0,Not reported,n.a.,n.a.,n.a.
8,Kellogg,Food,NaN,US,11 Mrd € (2019),https://crreport.kelloggcompany.com/ClimateChange,"Accélération des efforts à partir de 2015, ave...",4.0,6.0,-0.286717,...,90.0,83.2,-1.2,3.96,95.2,FF8939,Between +2°C and +3°C,-313.333333,5.372549,53.0
9,Mars,Food,NaN,US,32 Mrd € (2020),https://www.mars.com/sustainability-plan/healt...,"Accélération des efforts à partir de 2015, ave...",4.0,6.0,-0.219251,...,90.0,83.2,-0.5,3.4,98.0,FF8939,Between +2°C and +3°C,-420.0,6.0,58.0


In [79]:
df_final["Group"]

0                               Andros
1     Associated British Foods (alim.)
2                                  Bel
3                               Danone
4                              Ecotone
5                              Ferrero
6                        General Mills
7                                 Hero
8                              Kellogg
9                                 Mars
10                            Mondelez
11                              Nestlé
12                             Pepsico
13                              Henkel
14                                 P&G
15                            Unilever
16                   Colgate-Palmolive
17                                Coty
18                             L'Oréal
19                            Shiseido
20                          Beiersdorf
21                       S. C. Johnson
22                             Reckitt
23                                 GSK
24                   Johnson & Johnson
Name: Group, dtype: objec